# Курс ["Алгоритмы анализа данных"](https://gb.ru/lessons/177362)

# Урок 2. Масштабирование признаков. L1- и L2-регуляризация. Стохастический градиентный спуск

In [1]:
import numpy as np
import pandas as pd
import inspect
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика

## Задание 1.

### Измените функцию `calc_logloss` так, чтобы нули по возможности не попадали в `np.log` (как вариант - использовать `np.clip` или `np.where`).

In [3]:
from myalglib import MyLogReg

In [4]:
print(inspect.getsource(MyLogReg._calc_logloss))

    @staticmethod
    def _calc_logloss(y, y_pred):
        max_ = 10e+10 / (10e+10 + 1)  # 0.9999(9)
        min_ = 1 - max_  # 1.000000082740371e-11
        y_pred = np.clip(y_pred, min_, max_)
        err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
        err = np.sum(err)
        return err



## Задание 2

### Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss был минимальным. Покажите влияние гиперпараметров на ошибку алгоритма (оптимально здесь использовать критерий остановки обучения).

In [5]:
%time

logs = []
best_params = None
err_min = 1000
iterations = [5000, 4000, 3000, 2000, 1000]
alphas = [ 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6] 

for iteration in iterations:
    for alpha in alphas:
        log_r = MyLogReg(iter_num=iteration,alpha=alpha ,random_seed=42, scaler='standart')
        log_r.fit(X, y)
        err = min(log_r.err_logs)
        if err < err_min:
            err_min = err
            best_params = {'best iteration_num': iteration, 'best alpha': alpha}
        logs.append({'iteration_num': iteration, 'alpha': alpha, 'err': err})

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs


In [6]:
best_params

{'best iteration_num': 3000, 'best alpha': 0.1}

## Задание 3. 
### Создайте функцию `calc_pred_proba`, возвращающую предсказанную вероятность класса 1 (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`, на выходе - массив `y_pred_proba`).

In [7]:
print(inspect.getsource(MyLogReg.predict_proba))

    def predict_proba(self, x=None):
        """
        Расчёт вероятности
        """
        if x is None:
            x = self.x
        score = x.dot(self.w.T).flatten()
        return 1 / (1 + np.exp(-score))



In [8]:
log_r = MyLogReg(iter_num=best_params['best iteration_num'],
                 alpha=best_params['best alpha'], 
                 random_seed=42, 
                 scaler='standart')
log_r.fit(X, y)

In [9]:
log_r.predict_proba(X)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Задание 4.
### Создайте функцию `calc_pred`, возвращающую предсказанный класс (на вход подаются значения признаков `Х` и веса, которые уже посчитаны функцией `eval_LR_model`, на выходе - массив `y_pred`).

In [10]:
print(inspect.getsource(MyLogReg.predict))

    def predict(self, x, thr=0.5):
        proba = self.predict_proba(x)
        y_pred = np.zeros(proba.shape, dtype=bool)
        y_pred[proba > thr] = 1
        y_pred[proba <= thr] = 0
        self.y_pred = y_pred
        self.accuracy = len(y_pred[self.y == y_pred]) / len(y_pred)
        self.err_matrix = self.correlation_matrix(y_pred, self.y)
        self.precision = (len(y_pred[(self.y == y_pred) & (y_pred == 1)]) /
                        len(y_pred[(self.y == y_pred) & (y_pred == 1)]) +
                          len(y_pred[(self.y != y_pred) & (y_pred == 1)]))
        self.recall = (len(y_pred[(self.y == y_pred) & (y_pred == 1)]) /
                      (len(y_pred[(self.y == y_pred) & (y_pred == 1)]) +
                       len(y_pred[(self.y != y_pred) & (y_pred == 0)])))
        self.f1_score = (2 * self.precision * self.recall
                         / (self.precision + self.recall))
        return y_pred



In [11]:
log_r.predict(X)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

## Задание 5.

### Посчитайте accuracy, матрицу ошибок, precision и recall, а также F1-score.

In [12]:
log_r.err_matrix

,y = +1,y = -1
y_pred,,
a(x) = +1,5,5
a(x) = -1,0,0


In [13]:
log_r.accuracy, log_r.precision, log_r.recall

(0.5, 6.0, 1.0)

In [14]:
log_r.f1_score

1.7142857142857142

##  Задание 6.
### Могла ли модель переобучиться? Почему?

Да. Данных мало. Нет разбиения на тестовую и валидационную выборки

## Задание 7. (\*) 

### Создайте функции `eval_LR_model_l1` и `eval_LR_model_l2` с применением L1 и L2 регуляризации соответственно.

Задание уже выполнено как метод класса:

In [15]:
print(inspect.getsource(MyLogReg._penalty_value))

    def _penalty_value(self, penalty, w):
        """
        @return: L1, L2 penalty value or None
        """
        penalty_value = 0
        if penalty == 'l1':
            penalty_value = self.lambda_ * w / abs(w)
        if penalty == 'l2':
            penalty_value = 2 * self.lambda_ * w
        return penalty_value

